# Lab 2. Exploring Attach and Default Bearer Activation
***
# 1. Introduction
***
An attach procedure is carried out every time whenever a UE (User Equipment) needs to communicate to the Internet (PDN, Packet Data Network). The following procedure shows how to configure the Default EPS Bearer (the tunnel between UE and PDN-GW). A successful attachment to the network infrastructure via eNB leads to the ability for UEs to utilize the services offered by the LTE.

## 1.1 Learning Objectives
This lab focuses towards the following learning goals:

- Observe the protocols and packet communication in an Attach procedure.
- Understanding the separation of control and user plane communications.
- Comparative analysis of the observed attach procedure with theoretical concepts.



## 1.2 LTE Network
A model LTE architecture is provided in [Figure 1](#lte-arch) with the entities and the interfaces that are involved. 
<a id='lte-arch'></a>
<center>
    <figure>
        <img src= "Figures/LTE_net.png" alt="first" class="bg-primary mb-1" >
    </figure>
    <figcaption><b>Figure 1. Model LTE Network.</b></figcaption>
</center>

## 1.3 LTE Protocol Stack
The LTE protocol stacks can be segregated into User and Control Plane as shown in Figures 2 and 3 respectively. We will observe these protocols in packet level inspection performed in Section 3 of this lab.
### 1.3.1 User Plane
<a id='lte-stack-up'></a>
<center>
    <figure>
        <img src= "Figures/UserPlane.PNG" alt="first" class="bg-primary mb-1" >
    </figure>
    <figcaption><b>Figure 2. User Plane Protocol Stack in LTE</b></figcaption>
</center>

### 1.3.2 Control Plane
<a id='lte-stack-up'></a>
<center>
    <figure>
        <img src= "Figures/ControlPlane.PNG" alt="first" class="bg-primary mb-1" >
    </figure>
    <figcaption><b>Figure 3. Control Plane Protocol Stack in LTE</b></figcaption>
</center>
<br>
We will start with an introduction to `Wireshark` in order to carry out the capture analysis.

***
# 2. Wireshark tutorial and practical example.
***

A `pcap` file is a collection of captured packets in a network, having information about each individual packet which is sent over a given interface. 
This includes all the information in both header and payload.

Wireshark is a network-protocol analyzing tool which we are going to use in this lab activity to analyze `pcap` files.
You can download Wireshark at https://www.wireshark.org/download.html but it should already be available on the lab computers.

Moreover, the documentation for the wireshark software can be found at https://www.wireshark.org/docs/ that will serve as a reference as per requirement by the students.

## 2.1 Getting Started with Wireshark

As an introduction to Wireshark and the LTE attach procedure, in this first task of this lab you are going to analyze the ___first_attach.pcap___ file.

> Note: you can download the `pcap` file from this notebook, on the left pane just navigate to the correct folder, right-click and download.
> Alternatively, you can download it from the _github_ repository.

When you first open this file in Wireshark you will see something like this:

<img src= "Figures/Wireshark_first_time.png" alt="first" class="bg-primary mb-1" width="700px">

Here you have an overview of all the packets, and you can scroll through each individual packet 
and find layer information, flag values, protocols used and source/destination.

Since the file contains all the packets transmitted on the interfaces involved in the LTE attach procedure, there may be packets in the capture file which is not involved in the LTE attach procedure and thus is not relevant to us.

We can control which packets are shown by applying filters which can filter out certain protocols.
For example, during the LTE attach procedure the relevant protocols are S1-AP representing the S1 Control Plane and GTPv2, for the GPRS tunneling protocol, so you can use the filter `s1ap` or `gtpv2`.

> Some filter tips:
> * to apply several filters the `and` operator can be used
> * the `or` operator is available
> * to remove certain protocols the `!` operator is useful. 


## 2.2 Using the Flow Graph

`Wireshark` includes a useful functionality named "Flow Graph" that illustrates the packet flow between communicating nodes.

<img src= "Figures/flow_graph_1.png" alt="first" class="bg-primary mb-1" width="700px">

By clicking the "Statistics" drop-down, navigate to the "Flow Graph" option and choose it.
This will open the Flow Graph as seen below.

> Remember to choose the "Limit to Display Filter" option in the bottom left corner. 

<img src= "Figures/flow_graph_2.png" alt="first" class="bg-primary mb-1" width="700px">

In the next step, you will relate the obtained Flow Graph to the sequence diagram for the LTE attach procedure.

*** 
# 3. The Attach procedure
***
The attach procedure depicted in [Figure 4](#ATTACH_procedure) consists of 7 steps:
1) Random Access Procedure
2) Initial Connection Establishment
3) Authentication, Integrity and Ciphering
4) Update Location and UE Capability Procedure
5) Session Creation
6) Initial Context Setup
7) Attachment Completion

<a id='ATTACH_procedure'></a>
<center>
    <figure>
        <img src= "Figures/ATTACH_procedure.png" alt="first" class="bg-primary mb-1" width="1200px">
        <center><figcaption><b>Figure 4. Attach and default bearer activation.</b></figcaption></center>
    </figure>
</center>

<!-- To create a link to an image: [Sentence for the link](#id_of_pic) -->


> **NOTE:** first_attach.pcap doesn't provide the air interface capture.


Different steps constituting the attach procedure will be explored sequentially along with an analysis of the associated packets exchanges in the packet capture file in the following subsections.

## 3.1 Random Access Procedure


This is the first part that is performed, in order to allow the subscriber to access the network. The air interface is considered, i.e. the interface between eNodeB and UE. Once this procedure has been performed, the UE is known to the eNodeB and has been assigned a Cell Radio Network Temporary Identity (C-RNTI).


<a id='RandomAccessProcedure'></a>
<center>
    <figure>
        <img src= "Figures/RandomAccessProcedure.PNG" alt="first" class="bg-primary mb-1" width="250px">
        <figcaption><b>Figure 5. Random Access procedure.</b></figcaption>
    </figure>
</center>

- <u>Random Access Preamble</u>: UE requests to access the network, PRACH is considered;
- <u>Random Access Response</u>: eNodeB replies with information like Timing Advance Value, Uplink Grant Resource (eNodeB assigns initial resources to UE to use the PUSCH) and the Temporary Identifier for the UE (valid just within the cell).
- <u>RRC Connection Setup Request</u>: UE sends to eNodeB the received SAE (Service Architecture Evolution) Temporary Mobile Subscriber Identity (S-TMSI) and the reason why UE wants to connect to the network (Connection establishment cause). PUSCH is considered;
- <u>RRC Connection Setup</u>: acknowledgment by eNodeB that the previous message was successfully received. The message also contains configuration parameters like Uplink Power Control, the maximum number of Hybrid ARQ transmissions, etc...

## 3.2 Initial Connection Establishment

After the random access procedure is completed, the attaching setup will be performed.

---
### 3.2.1 Step 1
---
The UE starts the attaching procedure using the NAS layer, such that the layer between the UE and the MME.

<a id='AttachProcedure'></a>
<center>
    <figure>
        <img src= "Figures/AttachProcedure.PNG" alt="first" class="bg-primary mb-1" width="400px">
        <figcaption><b>Figure 6. Attach request.</b></figcaption>
    </figure>
</center>

- <u>RRC Connection Setup Complete + NAS Attach Request</u>: air interface is involved. The UE informs the eNodeB that the RRC connection is established, sending to the base station information like the old GUTI (Globally Unique Temporary Identity) (instead of the GUTI, it's possible to find the IMSI, i.e. International Mobile Subscriber Identity) and which MME it was previously connected (optional field, in case no previous connection was established, the field is not present). Here, we are considering a first attach request for a new UE so no optional information regarding previous MME is required. 
- <u>Attach Request</u>: S1 interface is involved (S1AP protocol). After the eNodeB selects the new MME, it forwards the UE attach request to that one. Further, it includes the PDN Connectivity Request message (to establish the EPS channel between UE and PDN-GW) and the Tracking Area Identify (TAI).

### 3.2.1.1 Exercise

Open the first_attach.pcap file. You can see something like this shown hereafter:

<center><img src= "Figures/PCAPAttachProcedure.PNG" alt="first" class="bg-primary mb-1" width="400px"></center>

#### <u>Question 3.2.1.1a</u>
Write down in *num_pck* the packet number, and in *interface* the name of the involved interface. (assuming air interface is not captured in the file)

In [1]:
S1APAttachRequest = {    
    "num_pck": ,
    "interface" : "",
}

## 3.3 Authentication, Integrity and Cyphering
Set of messages that are involved in order to authenticate and protect a subscriber. This is a very important part of the attaching procedure since it avoids the man-in-the-middle attack.

<a id='AuthenticationandCipheringProcedure1'></a>
<center>
    <figure>
        <img src= "Figures/AuthenticationandCipheringProcedure1.PNG" alt="first" class="bg-primary mb-1" width="500px">
        <figcaption><b>Figure 7. Attach request.</b></figcaption>
    </figure>
</center>


- <u>Authentication Data Request and Response</u>: S6 interface is involved (DIAMETER protocol). MME asks for authentication credentials to the HSS to authenticate and authorize the subscriber (UE), like the Key Access Security Management Entries (KASME);
- <u>NAS Authentication Request and Response</u>: air and S1 (S1AP protocol) interfaces are involved;
- <u>NAS Security Mode Command and Complete</u>: air and S1 (S1AP protocol) interfaces are involved;

### 3.3.1 Exercise

Return to the first_attach.pcap file.

<center><img src= "Figures/PCAPAuthenticationandCipheringProcedure1.PNG" alt="first" class="bg-primary mb-1" width="800px"></center>

#### <u>Question 3.3.1a</u> 
In which part of the capture file you can see this sequence? Fill the two variables below with the number of the first packet (*Authentication Data Request*) and the last one (*NAS Security Mode Complete*).

In [2]:
AuthenticationCiphering = {
    "From": ,
    "To": ,
}

#### <u>Question 3.3.1b</u> 
Compile the following gap with the IP address related to each involved entity.
> **NB**: pay attention to the corresponding interface.

In [3]:
S1Interface = {
    "eNodeB": "",
    "MME": "",
}
S6Interface = {
    "MME": "",
    "HSS": "",
}

#### <u>Question 3.3.1c</u> 
As you noticed, the IP of the UE is not present. In the following gap, explain the reason for it.

#### <u>Question 3.3.1d</u> 
Did you notice any authentication issue inside the procedure? If so, report it, providing the packet/s that is/are involved, the corresponding number of the packet/s and the entities involved (i.e. name of the entities). In addition, report the cause of the issue.
> **Hint**: see what happens in this phase, do you see any difference between the packet flow described in the theory?

## 3.4 Update Location and UE Capability Procedure
Now, let's continue with the analysis.

<a id='AuthenticationandCipheringProcedure2'></a>
<center>
    <figure>
        <img src= "Figures/AuthenticationandCipheringProcedure2.PNG" alt="first" class="bg-primary mb-1" width="500px">
        <figcaption><b>Figure 8. Update Location and UE Capability procedures.</b></figcaption>
    </figure>
</center>

- <u>Update Location Request and Answer</u>: S6 interface is involved. After the subscriber is authenticated, the MME confirms it to the HSS, sending the new location of the UE. You can find the IMSI of the UE under the AVP User-Name;
- <u>UE Capability Enquiry and Info</u>: air interface is involved. The eNodeB asks the UE the air interface functionalities, and UE replies with all supported radio technologies, such as the available frequency bands, the modulation scheme for each band, MIMO configuration, etc…</u>

### 3.4.1 Exercise

From the first_attach.pcap file, you can see something like as follows:

<center><img src= "Figures/PCAPAuthenticationandCipheringProcedure2.PNG" alt="first" class="bg-primary mb-1" width="500px"></center>

#### <u>Question 3.4.1a</u> 
Write down the packet numbers. Furthermore, report the IMSI of the UE.

In [4]:
UpdateLocationReqAns = {
    "UpdateLocationRequest": ,
    "UpdateLocationAnswer": ,
    "IMSI": ,
}

#### <u>Question 3.4.2a</u> 
Can you identify the MCC and MNC utilized in the capture file? Mention the operator and country they are utilized in?

## 3.5 Session Creation
This phase follows the Update Location Acknowledge message. In this part, the tunnel for the subscriber is created: 

<a id='SessionCreation'></a>
<center>
    <figure>
        <img src= "Figures/SessionCreation.PNG" alt="first" class="bg-primary mb-1" width="600px">
        <figcaption><b>Figure 9. Session Creation procedure.</b></figcaption>
    </figure>
</center>

- <u>Create Session Request and Response</u>: S11 and S5 interfaces are involved (GTPv2 protocol). MME initiates the default route establishment by asking the Serving-GW to create a GTP tunnel. Through the S-GW, the Create Session Request is forwarded to the PDN-GW, which replies with the assigned IP for the subscriber (Response). At this point, the tunnel (bearer) between Serving-GW and PDN-GW is established.
- <u>UE Capabilities indication</u>: S1 interface is involved. eNodeB informs the MME about the UE functionalities by sending an RRC message. In the S1 interface, the RRC message is encapsulated properly in the S1-AP packet.

### 3.5.1 Exercise

#### <u>Question 3.5.1a</u> 
Why is it important to establish a tunnel between the Serving-GW and the PDN-GW? Write down the advantage of this technique and why is needed.

From the first_attach.pcap file, you can see something like as follow:

<center><img src= "Figures/PCAPSessionCreation.PNG" alt="first" class="bg-primary mb-1" width="800px"></center>

#### <u>Question 3.5.1b</u> 
Write down the IP of PDN-GW and Serving-GW. Furthermore, provide the IP of the UE assigned by the PDN-GW. In *Answer*, provide which packet you considered to find the IP of the UE (name and number of the packet).

In [5]:
CreateSessionReqRes = {    
    "PDN_GW": "",
    "Serving_GW": "",
    "UE": "",
}

#### <u>Question 3.5.1c</u> 
By using the following filter, you can see the RRC message:

__`lte_rrc`__

Which is the UE Category? How many CAs (Carrier Aggregations) can be carried out? 

> **Hint 1:** you can find all the information under *UECapabilityInfoIndication* attribute.
>
> **Hint 2:** the maximum number that can be specified in the *ue-Category* attribute is 4. To find the corresponding value, you can use this [link](https://www.sharetechnote.com/html/LTE_Advanced_UE_Category.html).

<center><img src= "Figures/UECapabilitiesindication1.PNG" alt="first" class="bg-primary mb-1" width="600px"></center>

In [6]:
UECatCA = {    
    "ue_Category": ,
    "supportedBandCombination-r10": ,
}

#### <u>Question 3.5.1d</u> 
Describe the main benefits for the user through the Carrier Aggregation?

Let's take a look to the CA numbered 0 (Item 0).

<center><img src= "Figures/UECapabilitiesindication2.PNG" alt="first" class="bg-primary mb-1" width="400px"></center>

#### <u>Question 3.5.1e</u> 
Write down the E-UTRA Operating Band and the corresponding number of MIMO layers applied for Down Link.

In [7]:
UECapability = {
    'bandEUTRA': ,
    'DL_bandwidth': , #MHz
    'UL_bandwidth': , #MHz
    "DL_MIMO_layers": 
}

## 3.6 Establishing a Context in the Radio Network
This part takes place after the context in the core network is established.

<a id='EstablishingaContextintheRadioNetwork1'></a>
<center>
    <figure>
        <img src= "Figures/EstablishingaContextintheRadioNetwork1.PNG" alt="first" class="bg-primary mb-1" width="300px">
        <figcaption><b>Figure 10. Initial Context Setup procedure.</b></figcaption>
    </figure>
</center>

- <u>Initial Context Setup Request and Response</u>: S1 interface involved. MME responds back to the eNodeB with a packet containing three messages: S1AP Initial Context Setup Request, NAS Attach Accept and NAS Activate Default Bearer Request. Inside the Request, you can find information like the aggregate maximum bit rate for UL and DL, new GUTI assigned by the MME and other QoS parameters. Following that, the eNodeB sends the Response message to the MME as acknowledge, including the TEID (Tunnel Endpoint Identity, id used for the tunnel between Serving-GW and UE, i.e. E-RAB) of the eNodeB and the address of the eNodeB;
- <u>RRC Connection Reconfiguration and Reconfiguration Complete</u>: air interface is involved. It is sent to activate the default radio bearer and carries the Attach Accept message as NAS Payload. When the reconfiguration Complete is sent, this means that the RRC is configured (this message triggers the Initial Context Setup Request as discussed before);

## 3.7 Attachment Completion

<a id='EstablishingaContextintheRadioNetwork2'></a>
<center>
    <figure>
        <img src= "Figures/EstablishingaContextintheRadioNetwork2.PNG" alt="first" class="bg-primary mb-1" width="500px">
        <figcaption><b>Figure 11. Attach Complete and Modify Bearer procedures.</b></figcaption>
    </figure>
</center>


- <u>Attach Complete</u>: air and S1 interfaces are involved. When the UE is completely setup, it sends it for the MME an Activate Default Bearer Complete message through the NAS connection;
- <u>Modify Bearer Request and Response</u>: S11 interface is involved (GTPv2 protocol). The MME informs the Serving-GW about the TIED used by the considered eRAB. Afterwards, the Response is carried out by the S-GW, the buffered downlink data (if it is present) can be sent through the eRAB.

### 3.7.1 Exercise

From the first_attach.pcap file, you can see something like as follows:

<center><img src= "Figures/PCAPEstablishingaContextintheRadioNetwork1.PNG" alt="first" class="bg-primary mb-1" width="700px"></center>

#### <u>Question 3.7.1a</u> 
Write down the aggregated maximum bit rate associated to the UE, both for UL and DL. Furthermore, report the number of the packet where you found this information.

In [8]:
uEaggregateMaximumBitRate = {
    "uEaggregateMaximumBitRateDL": , #bit/s
    "uEaggregateMaximumBitRateUL": , #bit/s
    "num_pck": ,
}

#### <u>Question 3.7.1b</u> 
Describe one possible factor to increase the aggregated maximum bit rate (AMBR) for uplink?

#### <u>Question 3.7.1c</u> 
What could be the limiting factor in determining the AMBR of a UE?

-------